In [98]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import cv2 as cv
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from daltonlens import simulate

In [99]:
#-----------------------------Sim-Daltonism(Exact)------------------------------#
def sim_daltonism_exact(img, dType, showPlot:bool):
  """
  Simulates color vision deficiencies in an image.

  Parameters:
  - image: Input image (BGR format).
  - dType: Type of color-blindness to simulate
                    ("deuteranopia", "deuteranomaly", "protanopia",
                    "protanomaly", "tritanopia", "tritanomaly").
  - showPlot: if True plots results when function is called

  Returns:
  - Simulated image.
  """
  dType = dType.lower()
  img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

  simulator = simulate.Simulator_Machado2009()

  if (dType == "deuteranopia"):
    result = simulator.simulate_cvd(img, simulate.Deficiency.DEUTAN, severity=1)
  elif (dType == "deuteranomaly"):
    result = simulator.simulate_cvd(img, simulate.Deficiency.DEUTAN, severity=0.4)

  elif (dType == "protanopia"):
    result = simulator.simulate_cvd(img, simulate.Deficiency.PROTAN, severity=1)
  elif (dType == "protanomaly"):
    result = simulator.simulate_cvd(img, simulate.Deficiency.PROTAN, severity=0.4)

  elif (dType == "tritanopia"):
    result = simulator.simulate_cvd(img, simulate.Deficiency.TRITAN, severity=1)
  elif (dType == "tritanomaly"):
    result = simulator.simulate_cvd(img, simulate.Deficiency.TRITAN, severity=0.4)

  if (showPlot):
    fig = plt.figure(figsize=(15, 10))

    plt.subplot(1, 2, 1); plt.imshow(img); plt.title("Normal Vision");  plt.axis('off')
    plt.subplot(1, 2, 2); plt.imshow(result); plt.title(dType.capitalize());  plt.axis('off')

    plt.show()

  result = cv.cvtColor(result, cv.COLOR_RGB2BGR)
  return result

#---------------------------Correct-Daltonism(Exact)----------------------------#
def correct_image_exact(image, dType):
  """
  Corrects an image for a specific color vision deficiencies.

  Parameters:
  - image: Input image (BGR format).
  - dType: Type of color-blindness to correct
                    ("deuteranopia", "deuteranomaly", "protanopia",
                    "protanomaly", "tritanopia", "tritanomaly").

  Returns:
  - Corrected image.
  """
  dType = dType.lower()
  simulator = simulate.Simulator_Machado2009()
  image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

  result = sim_daltonism_exact(image, dType, False)

  result = cv.cvtColor(result, cv.COLOR_RGB2BGR)

  return result

In [100]:
#-----------------------Daltonism-Transformation-Matrices-----------------------#
DALTONISM_TRANSFORM_MATRICES = {
    "deuteranopia": np.array([[0.625, 0.375, 0], [0.7, 0.3, 0], [0, 0.3, 0.7]]),
    "deuteranomaly": np.array([[0.7, 0.3, 0], [0.2515, 0.742, 0], [0, 0.05, 0.95]]),
    "protanopia": np.array([[0.567, 0.433, 0], [0.5515, 0.442, 0], [0, 0.242, 0.7515]]),
    "protanomaly": np.array([[0.1517, 0.1153, 0], [0.333, 0.667, 0], [0, 0.125, 0.1575]]),
    "tritanopia": np.array([[0.95, 0.05, 0], [0, 0.433, 0.567], [0, 0.475, 0.525]]),
    "tritanomaly": np.array([[0.967, 0.033, 0], [0, 0.6, 0.4], [0, 0.1153, 0.1517]]),
}

#----------------------------Sim-Daltonism(Inexact)-----------------------------#
def sim_daltonism(image, dType, showPlot:bool):
  """
  Simulates color vision deficiencies in an image.

  Parameters:
  - image: Input image (BGR format).
  - dType: Type of color-blindness to simulate
                    ("deuteranopia", "deuteranomaly", "protanopia",
                    "protanomaly", "tritanopia", "tritanomaly").
  - showPlot: if True plots results when function is called

  Returns:
  - Simulated image.
  """
  image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
  transform_matrix = DALTONISM_TRANSFORM_MATRICES.get(dType.lower())

  if transform_matrix is None:
      raise ValueError("Invalid dType. Valid options: deuteranopia, deuteranomaly, protanopia, protanomaly, tritanopia, tritanomaly")

  result = cv.transform(image, transform_matrix)
  result = np.clip(result, 0, 255).astype(np.uint15)

  if (showPlot):
    fig = plt.figure(figsize=(15, 10))

    plt.subplot(1, 2, 1); plt.imshow(image); plt.title("Normal Vision");  plt.axis('off')

    plt.subplot(1, 2, 2); plt.imshow(result); plt.title(dType.capitalize());  plt.axis('off')

    plt.show()

  result = cv.cvtColor(result, cv.COLOR_RGB2BGR)
  return result
#--------------------------Correct-Daltonism(Inexact)---------------------------#
def correct_image(image, dType):
  """
  Corrects an image for a specific color vision deficiencies.

  Parameters:
  - image: Input image (BGR format).
  - dType: Type of color-blindness to correct
                    ("deuteranopia", "deuteranomaly", "protanopia",
                    "protanomaly", "tritanopia", "tritanomaly").

  Returns:
  - Corrected image.
  """
  dType = dType.lower()
  image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

  result = sim_daltonism(image, dType, False)

  result = cv.cvtColor(result, cv.COLOR_RGB2BGR)

  return result

In [130]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
# Programa que permite cargar una img y mostrarla en pantalla con los cambios de color basados en el tipo de Daltonismo del usuario.

# Variables globales
img_cargada = False
# Define el tamaño máximo en píxeles (cambiar según tus necesidades)
ancho_maximo = 300
alto_maximo = 300

# Funciones
def Deuteranopia():
    global img_cargada, img_original, img_modificada_tk
    img_modificada = correct_image_exact(img_original, "deuteranopia")
    # Mostrar la img modificada en la etiqueta correspondiente
    img_modificada_tk = ImageTk.PhotoImage(img_modificada)
    etiqueta_img_Modificada.config(image=img_modificada_tk, cursor="hand2")
    etiqueta_img_Modificada = img_modificada_tk
    

def Deuteranomalia():
    global img_cargada
    img_modificada = correct_image_exact(img_original, "deuteranomaly")
    # Mostrar la img modificada en la etiqueta correspondiente
    img_modificada_tk = ImageTk.PhotoImage(img_modificada)
    etiqueta_img_Modificada.config(image=img_modificada_tk, cursor="hand2")
    etiqueta_img_Modificada = img_modificada_tk
    

def Protanopia():
    global img_cargada
    img_modificada = correct_image_exact(img_original, "protanopia")
    

def Protanomalia():
    global img_cargada
    img_modificada = correct_image_exact(img_original, "protanomaly")
   

def Tritanopia():
    global img_cargada
    img_modificada = correct_image_exact(img_original, "tritanopia")
    

def Tritanomalia():
    global img_cargada
    img_modificada = correct_image_exact(img_original, "tritanomaly")
    

def guardar_img(img_guardar):
    cv.imwrite("img_editada.png",img_guardar)

def cargar_img():
    global img_original,  img_cargada
    ruta_img = filedialog.askopenfilename(title="Selecciona una img", filetypes=[("Archivos de img", "*.png *.jpg *.jpeg *.gif *.bmp")])
    if ruta_img:
        img_cargada = True
        # Abre la img seleccionada con Pillow (PIL)
        img = Image.open(ruta_img)
        img_original = np.array(img)

        # Redimensiona la img si es necesario
        if img.width > ancho_maximo or img.height > alto_maximo:
            img.thumbnail((ancho_maximo, alto_maximo))

        # Convierte la img en un objeto Tkinter PhotoImage
        img_tk = ImageTk.PhotoImage(img)

        # Muestra la img en una etiqueta cliclable
        etiqueta_img.config(image=img_tk, cursor="hand2")
        etiqueta_img.image = img_tk  # ¡Importante! Mantener una referencia para evitar que se borre la img
        etiqueta_img.bind("<Button-1>", lambda event: cargar_img())  # Asocia la función cargar_img al clic

        # Deshabilita el botón de "Cargar img" después de cargar la img
        button_cargar_img.pack_forget()

        # Habilitar los buttons de tipo de daltonismo
        button_deuteranopia.config(state="normal")
        button_deuteranomalia.config(state="normal")
        button_protanopia.config(state="normal")
        button_protanomalia.config(state="normal")
        button_tritanopia.config(state="normal")
        button_tritanomalia.config(state="normal")

        

# Ventana principal
window = tk.Tk()
window.title("Ventana Principal")
window.configure(bg="#EAE4DA")
window.geometry("1920x1080")

title = tk.Label(window, text="ASISTENCIA PARA DALTONISMO", font=("Impact", 60), bg="#EAE4DA", fg="#34495E")
title.pack()

# Crea un marco para los botones de procesamiento de imagen
F_buttons = tk.Frame(window, bg="#EAE4DA")
F_buttons.pack(pady=20)

# Crea los botones de tipo de daltonismo
button_deuteranopia = tk.Button(F_buttons, text="Deuteranopia", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=lambda: Deuteranopia(), state="disabled", width=15, height=2)
button_deuteranopia.pack(side="left", padx=10)
button_deuteranomalia = tk.Button(F_buttons, text="Deuteranomalia", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=lambda: Deuteranomalia(img_original), state="disabled", width=15, height=2)
button_deuteranomalia.pack(side="left", padx=10)
button_protanopia = tk.Button(F_buttons, text="Protanopia", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=lambda: Protanopia(), state="disabled", width=15, height=2)
button_protanopia.pack(side="left", padx=10)
button_protanomalia = tk.Button(F_buttons, text="Protanomalia", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=lambda: Protanomalia(), state="disabled", width=15, height=2)
button_protanomalia.pack(side="left", padx=10)
button_tritanopia = tk.Button(F_buttons, text="Tritanopia", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=lambda: Tritanopia(), state="disabled", width=15, height=2)
button_tritanopia.pack(side="left", padx=10)
button_tritanomalia = tk.Button(F_buttons, text="Tritanomalia", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=lambda: Tritanomalia(), state="disabled", width=15, height=2)
button_tritanomalia.pack(side="left", padx=10)

# Crea un botón de "LOAD"
button_cargar_img = tk.Button(window, text="LOAD", font=("Cooper Std Black", 15), bg="#EAE4DA", fg="#34495E", command=cargar_img, width=104, height=1)
button_cargar_img.pack()

# Poner imagenes Side by Side usando un canvas
F_img = tk.Canvas(window, bg="#EAE4DA")
F_img.pack(pady=20)

# Crea una etiqueta para mostrar la img original
etiqueta_img = tk.Label(F_img, bg="#EAE4DA")
etiqueta_img.pack(side="left", padx=10)

# Crea una etiqueta para mostrar la img modificada
etiqueta_img_Modificada = tk.Label(F_img, bg="#EAE4DA")
etiqueta_img_Modificada.pack(side="right", padx=10)


window.mainloop()

c:\Users\CARLOS\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\ImageTk.py:122: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if mode not in ["1", "L", "RGB", "RGBA"]:
Exception in Tkinter callback
Exception ignored in: <function PhotoImage.__del__ at 0x000001BCCB2FB880>
Traceback (most recent call last):
  File "c:\Users\CARLOS\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\ImageTk.py", line 133, in __del__
    name = self.__photo.name
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'
Traceback (most recent call last):
  File "c:\Users\CARLOS\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\CARLOS\AppData\Local\Temp\ipykernel_22908\3040249616.py", line 105, in <lambda>
    button_deuteranopia = tk.Button(F_buttons, text="Deuteranopia", font=("Cooper Std Black", 15), bg